In [ ]:
!git clone https://github.com/andrewaeva/DGA
!pip install wordsegment
!pip install pyenchant
!apt install hunspell
!apt install hunspell-es
!apt install hunspell-de-de
!apt install hunspell-fr
!apt install hunspell-it
!apt install hunspell-uk
!apt-get install enchant-2
!pip install syllables

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = """cryptolocker    1
zeus            2
pushdo          3
rovnix          4
tinba           5
matsnu          7
ramdo           8"""

# Split the data into lines
lines = data.split('\n')

# Extract names and labels into separate lists
names = []
labels = []

for line in lines:
    parts = line.split()
    if len(parts) == 2:
        name, label = parts
        names.append(name)
        labels.append(int(label))

lines = []
print(len(names))
for i in range(len(names)):
    with open("DGA/dga_wordlists/"+names[i]+".txt", "r") as f:
        lines += [next(f).strip()+' '+str(labels[i]) for _ in range(30000)]


with open("DGA/all_legit.txt") as f:
    lines += [next(f).strip() for _ in range(300123)]

with open("/content/drive/MyDrive/dataset/datset1.txt", "w") as f:
    f.write('\n'.join(lines))


7


In [ ]:

# features



# domain length
def getDomainLength(domain):
    return len(domain)


from wordsegment import load, segment
import wordsegment

def domain2list(domain):
    if (len(domain)>0):
        s = wordsegment.segment(domain)
        return s
    return []
# number of length
def getNumOfHyphen(domain):
    return domain.count("-")
# number of numeric tokens
import re
def getNumOfDigit(domain):
    n = re.findall(r"\d+",domain)
    return len(n)

# rare of word
import nltk
from nltk.corpus import brown
from nltk.probability import FreqDist
nltk.download('brown')
def getRareOfWords(domain_word_list):
    randomWordCount = 0
    rareWordCount = 0
    commonWordCount = 0
    for i in domain_word_list:
        if domain_freq.get(i) is None:
            randomWordCount += 1
        elif domain_freq.get(i) < 10000:
            rareWordCount +=1
        else:
            commonWordCount +=1
    global rareness
    rareness = [randomWordCount, rareWordCount, commonWordCount]
    return rareness

def loadRareness(domain_word_list):
    getRareOfWords(domain_word_list)

# common ratio
def getCommonRatio(domain_word_list):
    return rareness[2] / len(domain_word_list)

# rare ratio
def getRareRatio(domain_word_list):
    return rareness[1] / len(domain_word_list)
# domain type
def isDomainTypeIllicit(domaintype):
    if domaintype in ["com","org","ru","net","biz"]:
        return True
    return False

import enchant


def getMeaningRatio(domain_word_list: list) -> list:
    mean = 0
    mispelled = 0
    for i in domain_word_list:
        if check_dict.check(i) or check_dict.check(i.capitalize()):
            mean+=1
        else: mispelled += 1
    return [mean/len(domain_word_list), mispelled/len(domain_word_list)]

# no random
def getNumberRandomWord():
    return rareness[2]


# calculate weight value of each letter

qwerty_keyboard = {
    '1': 3, '2': 3, '3': 3, '4': 3, '5': 3, '6': 3, '7': 3, '8': 3, '9': 3, '0': 3, '-':3, '_': 4,
    'q': 2, 'w': 2, 'e': 2, 'r': 2, 't': 2, 'y': 2, 'u': 2, 'i': 2, 'o': 2, 'p': 2,
    'a': 1, 's': 1, 'd': 1, 'f': 1, 'g': 1, 'h': 1, 'j': 1, 'k': 1, 'l': 1,
    'z': 2, 'x': 2, 'c': 2, 'v': 2, 'b': 2, 'n': 2, 'm': 2
}
def calculateWeightLetters(domain):
    weight = 0
    for i in domain:
        a = qwerty_keyboard.get(i)
        if type(a) is not int:
          print(domain)
          break
        weight += a
    return weight

def calculateConsecutive(domain: str):
    points = 0
    consecutive_count = 0
    for i in range(len(domain)):
        if i > 0 and qwerty_keyboard.get(domain[i]) == qwerty_keyboard.get(domain[i-1]):
            consecutive_count += 1
        else:
            consecutive_count = 1
        if consecutive_count == 3:
            points += 1

    return points


# digraph frequency
from collections import Counter

def calculate_digraph_frequency(text):
    digraphs = [text[i:i+2] for i in range(len(text)-1)]
    return Counter(digraphs)
def calculate_trigraph_frequency(text):
    trigraphs = [text[i:i+3] for i in range(len(text)-2)]
    return Counter(trigraphs)
def calculate_char_frequency(text):
    chars = [text[i] for i in range(len(text))]
    return Counter(chars)

# difficult of word
def getWeightDifficult(domain, domain_word_list):
    domain_len = getDomainLength(domain)
    w = getNumOfDigit(domain) / domain_len
    h = getNumOfHyphen(domain) / domain_len

    dgraph = calculate_digraph_frequency(domain)
    trigraph = calculate_digraph_frequency(domain)
    s_dgraph = sum([value for key, value in dgraph.items()])
    s_trigraph = sum([value for key, value in trigraph.items()])

    d = ( s_dgraph + s_trigraph) / domain_len
    list_dom = domain_word_list
    r = rareness[1] / len(list_dom)
    l = 0
    if domain_len > 10:
        l = domain_len - 10
    weight_letter = calculateWeightLetters(domain)
    y = calculateConsecutive(domain)

    weight = w + h + d + r + l + weight_letter - y
    return weight

# get readable index
import re
import syllables

def getReadableIndex(domain_word_list):
    text = ' '.join(domain_word_list)
    flesch_reading_ease = 206.835 - 1.015 * (len(re.findall(r'\b\w+\b', text))) - \
                    84.6 * (sum(syllables.estimate(word) for word in re.findall(r'\b\w+\b', text)) / len(re.findall(r'\b\w+\b', text)))
    return flesch_reading_ease

# get frequency
def getDCharD(domain):
    dgraph = calculate_digraph_frequency(domain)
    trigraph = calculate_digraph_frequency(domain)
    chars = calculate_char_frequency(domain)
    s_dgraph = sum([value for key, value in dgraph.items()])
    s_trigraph = sum([value for key, value in trigraph.items()])
    s_chars = sum([value for key, value in chars.items()])
    s = s_chars + s_trigraph + s_dgraph
    return s/len(domain)

# get char frequen in alexa-top 1 mil

freq_letter = {
    'a': 7.07, 'b': 2.05, 'c': 7.06, 'd': 2.67, 'e': 7.89,
    'f': 1.32, 'g': 2.36, 'h': 2.01, 'i': 5.71, 'j': 0.41,
    'k': 1.49, 'l': 3.90, 'm': 6.31, 'n': 5.05, 'o': 10.08,
    'p': 2.46, 'q': 0.14, 'r': 6.04, 's': 5.14, 't': 5.19,
    'u': 2.97, 'v': 1.11, 'w': 0.94, 'x': 0.47, 'y': 1.26,
    'z': 0.57, '0': 0.11, '1': 0.16, '2': 0.15, '3': 0.09,
    '4': 0.10, '5': 0.06, '6': 0.05, '7': 0.05, '8': 0.06,
    '9': 0.05, '-': 0.82, '_': 6.63
}

def getCharFreq(domain):
    freq = 0
    for i in domain:
        freq+=freq_letter[i]
    return freq


# calculate entropy
import math


def getEntropy(domain):
    "Calculates the Shannon entropy of a string"
    # get probability of chars in string
    prob = [ float(domain.count(c)) / len(domain) for c in dict.fromkeys(list(domain)) ]
    # calculate the entropy
    entropy = - sum([ p * math.log(p) / math.log(2.0) for p in prob ])
    return entropy



def Load():
    global words, domain_freq
    words = [word.lower() for word in brown.words() if word.isalpha()]
    domain_freq = FreqDist(words)

    global check_dict
    check_dict = enchant.Dict("en-US")
    for i in enchant.list_languages():
        check_dict.add(i)

    wordsegment.load()



[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [ ]:
def get_domain(domain):
    a = domain.split('.')
def getFeature(domain, label, domaintype):
    domain_word_list = domain2list(domain)
    meaning = getMeaningRatio(domain_word_list)
    loadRareness(domain_word_list)
    features = []
    features.append(domain)
    features.append(getDomainLength(domain))
    features.append(len(domain_word_list))
    features.append(getNumOfHyphen(domain))
    features.append(getNumOfDigit(domain))
    features.append(getRareRatio(domain_word_list))
    features.append(getCommonRatio(domain_word_list))
    features.append(meaning[0])
    features.append(meaning[1])
    features.append(getNumberRandomWord())
    features.append(getReadableIndex(domain_word_list))
    features.append(getWeightDifficult(domain, domain_word_list))
    features.append(isDomainTypeIllicit(domaintype))
    features.append(getDCharD(domain))
    features.append(getCharFreq(domain))
    features.append(getEntropy(domain))
    features.append(label)

    return features

# 3. Dataset


In [ ]:
import pandas as pd


main_data = pd.read_csv('/content/drive/MyDrive/dataset/datset1.txt',sep =' ', header= None)
main_data.columns = ['domain','type']

# 4. Feature collection

In [ ]:


feature_names = ['Domain','DomainLength', 'NoOfDomainWords', 'NoOfHyphen',
                 'NoOfNumericToken', 'RareRatio', 'CommonRatio',
                 'HasMeaningRatio', 'MisspelledRatio', 'NoOfRandom',
                 'Readable', 'isDifficult', 'Dtype', 'DCharD',
                 'CharFreq', 'Entropy','Label'
                 ]

domain_features = []
Load()
for i in range(510122):
    a = main_data['domain'][i].split('.')
    domain = a[0]
    domaintype = a[1]
    label = main_data['type'][i]
    # i have changed label here. You can keep the label value or whatever you want here.
    if(int(label) > 1):
        label = 1
    print("Collecting...", i, end="\r")
    domain_features.append(getFeature(domain,label,domaintype))

domain_data = pd.DataFrame(domain_features, columns=feature_names)

domain_data.to_csv('/content/drive/MyDrive/dataset/data0_atm.csv')



# Training

In [ ]:
data = domain_data.drop(['Domain'],axis=1).copy()
data = data.sample(frac=1).reset_index(drop=True)
y = data['Label']
X = data.drop('Label',axis=1)

## Decision tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.36, random_state = 12)




In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=5)
tree.fit(X_train, y_train)

y_test_tree = tree.predict(X_test)

acc_test_tree = accuracy_score(y_test,y_test_tree)

print("Decision Tree: Accuracy on test Data: {}".format(acc_test_tree))
conf_matrix = confusion_matrix(y_test, y_test_tree)

tn, fp, fn, tp = conf_matrix.ravel()

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_test_tree)
recall = recall_score(y_test, y_test_tree)
f1 = f1_score(y_test, y_test_tree)
print("False Positives: {}".format(fp))
print("False Negatives: {}".format(fn))
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))
print("F1 Score: {}".format(f1))

Decision Tree: Accuracy on test Data: 0.9640445644834571
False Positives: 5134
False Negatives: 1469
Precision: 0.935
Recall: 0.981
F1 Score: 0.958


## k-NN algorithm

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_test_knn = knn.predict(X_test)

acc_test_knn = accuracy_score(y_test, y_test_knn)

print("K-Nearest Neighbors: Accuracy on test data: {}".format(acc_test_knn))
conf_matrix = confusion_matrix(y_test, y_test_tree)

tn, fp, fn, tp = conf_matrix.ravel()

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_test_tree)
recall = recall_score(y_test, y_test_tree)
f1 = f1_score(y_test, y_test_tree)
print("False Positives: {}".format(fp))
print("False Negatives: {}".format(fn))
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))
print("F1 Score: {}".format(f1))

K-Nearest Neighbors: Accuracy on test data: 0.9654657925115985
False Positives: 5134
False Negatives: 1469
Precision: 0.9354863030912289
Recall: 0.9806494105249292
F1 Score: 0.9575356120775588


## ANN 

In [ ]:
# Artificial Neural Network
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

ann = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
ann.fit(X_train, y_train)

y_test_ann = ann.predict(X_test)

acc_test_ann = accuracy_score(y_test, y_test_ann)

print("Artificial Neural Network: Accuracy on test data: {:.3f}".format(acc_test_ann))
conf_matrix = confusion_matrix(y_test, y_test_tree)

tn, fp, fn, tp = conf_matrix.ravel()

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_test_tree)
recall = recall_score(y_test, y_test_tree)
f1 = f1_score(y_test, y_test_tree)
print("False Positives: {}".format(fp))
print("False Negatives: {}".format(fn))
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))
print("F1 Score: {}".format(f1))

Artificial Neural Network: Accuracy on test data: 0.975
False Positives: 5134
False Negatives: 1469
Precision: 0.9354863030912289
Recall: 0.9806494105249292
F1 Score: 0.9575356120775588


## SVM algorithm

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm = SVC(kernel='poly')
svm.fit(X_train, y_train)

y_test_svm = svm.predict(X_test)

acc_test_svm = accuracy_score(y_test, y_test_svm)

print("Support Vector Machine: Accuracy on test data: {:.3f}".format(acc_test_svm))
conf_matrix = confusion_matrix(y_test, y_test_tree)

tn, fp, fn, tp = conf_matrix.ravel()

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_test_tree)
recall = recall_score(y_test, y_test_tree)
f1 = f1_score(y_test, y_test_tree)
print("False Positives: {}".format(fp))
print("False Negatives: {}".format(fn))
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))
print("F1 Score: {}".format(f1))



Support Vector Machine: Accuracy on test data: 0.865
False Positives: 5134
False Negatives: 1469
Precision: 0.9354863030912289
Recall: 0.9806494105249292
F1 Score: 0.9575356120775588


In [ ]:
print(acc_test_svm)

NameError: ignored